In [ ]:
# the Python Requests package will allow us to send HTTP requests to get HTML files
import requests

# the GET method indicates that you’re trying to get or retrieve data from a specified resource. 
# to make a GET request, invoke requests.get()
from requests import get

# Beautiful Soup is a Python library for pulling data out of HTML and XML files
from bs4 import BeautifulSoup

# pandas is a Python data analysis library
import pandas as pd

# NumPy is a Python library used for working with large, multi-dimensional arrays and matrices
import numpy as np

# Datetime is a module that supplies classes for manipulating dates and times
from datetime import datetime

# The os module provides a portable way of using operating system dependent functionality
import os

# re can be used to work with Regular Expressions
import re

# The os.path module implements some useful functions on pathnames
import os.path as path

In [ ]:
# look in working directory for CSV file where existing data is stored and read as dataframe
# if this file does not exist, create empty dataframe with columns corresponding to required information
if path.exists('canadacannabisdispensary.csv'):
   existing_data = pd.read_csv('canadacannabisdispensary.csv')
else:
   columns = ['date', 'product','price', 'quantity']
   existing_data = pd.DataFrame(columns = columns)

In [ ]:
existing_data

As of April 2022, The Canada Cannabis Dispensary website has been taken down since March 2022, and is currently not accessible by this URL. This block of code will throw a connection error unless the website returns, but should be used if the website returns. If a new website returns under a different URL, the URL should be updated.

In [ ]:
# the URL we want to obtain data from
url = 'https://www.canadacannabisdispensary.co/product-category/aaa/'

# store scraped data in lists
all_names = []
all_prices = []
all_quantities = []
date = []

results = requests.get(url)

# parse web content into Python-readable format
soup = BeautifulSoup(results.content, "html.parser")

# from inspecting the HTML, information we need is stored here for each product
# the find_all() function will find every instance of matching tags and filters from the soup
products = soup.find_all('a', class_="woocommerce-LoopProduct-link woocommerce-loop-product__link")

# iterate through each product and obtain data from each
for product in products:  
    # obtain where the name of each product is stored
    name = product.find('h2', class_='woocommerce-loop-product__title').text
    
    # obtain where the URL of each individual product page is stored
    product_page = product.get('href')
    
    # obtain HTML for individual product pages
    # individual product pages contain price and quantity information
    product_page_results = requests.get(product_page)
    product_page_soup = BeautifulSoup(product_page_results.content, "html.parser")
    
    # obtain where the price of each product is stored
    price = product_page_soup.find('span', class_='woocommerce-Price-amount amount').text
    
    # obtaining the correct corresponding quantity for a given price requires sorting through all quantity options
    # this is necessary for calculating a price per quantity for each product if desired
    quantities = product_page_soup.find_all('option')
    
    # list to store all quantity options available for a given product
    stripped_quantities = [] 
    
    # if quantity options exist, sort through each option and add to list of options
    if quantities:
        for quantity in quantities:
            # find all number quantity options listed on product page by removing non-numeric characters from each
            strip = re.findall(r"[-+]?(?:\d*\.\d+|\d+)", quantity.text)
            if strip:
                stripped_quantities.append(float(strip[0]))
    
    print(stripped_quantities)
    
    # if stripped_quantities is non-empty, add the information to the quantity column
    if stripped_quantities:
        # the price we need corresponds to the smallest quantity available for sale
        all_quantities.append(min(stripped_quantities))
    else:
        all_quantities.append('No quantity options')

    # obtain the date the script is being run
    current_date = datetime.today().strftime('%Y-%m-%d')

    all_names.append(name)
    all_prices.append(price)
    date.append(current_date)

In [ ]:
print(all_names)
print(all_prices)
print(all_quantities)
print(date)

In [ ]:
# create dataframe using new data
new_data = pd.DataFrame(
    {'date': date,
     'product': all_names,
     'price': all_prices,
     'quantity': all_quantities
    })

In [ ]:
new_data

In [ ]:
# join original and new data by column
joined_data = pd.concat([existing_data, new_data])

In [ ]:
joined_data

In [ ]:
# ensure duplicates are removed in case data is added twice
final_data = joined_data.drop_duplicates(subset=['date', 'product'], keep = 'last', inplace=False)

In [ ]:
final_data

In [ ]:
# define filename for data
output_path = 'canadacannabisdispensary.csv'

# overwrite previous file with new file containing previous and new data
df.to_csv(output_path, index = False)